# <font color='black'>НИС: регрессионный анализ, 2025 </font>
## <font color='black'> Практическое занятие: Гетероскедастичность. Нетипичные наблюдения </font>
В рамках данного практического занятия мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Возьмем такой массив, так как мы пока не знакомились с моделями для анализа панельных данных.

Стоит отметить, что авторы изучают взаимосвязь уровня коррупции (является откликом в регрессионной модели) и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Мы протестируем данную гипотезу на практическом занятии 2, когда познакомимся с регрессионными моделями с переменными взаимодействия. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Приведен к непрерывной шкале от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения: World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels as sm
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import shapiro
import seaborn as sns
import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.lines as mlines

from statsmodels.tools.tools import add_constant

from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.diagnostic import het_goldfeldquandt

from statsmodels.stats.outliers_influence import outlier_test

Откроем массив данных для репликации результатов исследования: lab1.dta

In [ ]:
lab1 = pd.read_stata('lab1.dta')
lab1 = lab1.dropna()

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit()
print(m1.summary())

Проверим, можно ли говорить про гомоскедастичность (здравый смысл подсказывает, что надеяться не стоит), но тем не менее, реализуем определенные диагностики. Для начала построим график, показывающий, как связаны остатки в квадрате и одна из объясняющих переменных - к примеру, ключевой предиктор (dem).

In [ ]:
fitted = m1.predict()
residuals_sq = m1.resid**2
fig, ax = mpl.subplots()
sns.scatterplot(x = lab1['dem'], y = residuals_sq)
ax.set_xlabel( "Индекс демократии")
ax.set_ylabel( "Остатки в квадрате")

Можно не рассматривать объясняющие переменные по отдельности, а  построить график, показывающий, как связаны остатки в квадрате и предсказанные значения по модели ($\hat{y}$ как линейная комбинация предикторов)

In [ ]:
fitted = m1.predict()
residuals_sq = m1.resid**2
fig, ax = mpl.subplots()
sns.scatterplot(x = fitted, y = residuals_sq)
ax.set_xlabel( "Предсказанные значения отклика")
ax.set_ylabel( "Квадраты остатков")

Есть и формальные тесты для проверки гипотезы о гомоскедастичности. К примеру, тест Уайта. Давайте его реализуем и проинтепретируем результаты на основе p-value.

In [ ]:
X = lab1[["dem", "fp", "stab", "loggdppc", "britcol"]]
X = add_constant(X)

white_test = het_white(m1.resid, X)
white_test

stat, p_value = white_test[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

In [ ]:
white_test1 = het_white(m1.resid, m1.model.exog)
stat, p_value = white_test1[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

Как мы уже обсуждали, можно использовать тест Бреуша-Пагана, предполагающий более экономную спецификацию вспомогательной модели по сравнению с тестом Уайта (остатки в квадрате регрессируются на исходные объясняющие переменные, не учитываются квадратичные эффекты и совместные эффекты).

In [ ]:
bp_test = het_breuschpagan(m1.resid, m1.model.exog)
stat, p_value = bp_test[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

Кроме этого, мы можем рассмотреть частный случай гетероскедастичности. Мы можем предположить, что вариация остатков монотонно возрастает/убывает с ростом предиктора. Для иллюстрации возьмем предиктор "loggdppc"

Отвержение нулевой гипотезы говорит не в целом о гетероскедастичности, а наличии определенного паттерна, задающего различия в вариации остатков

В этом тесте возможно задать и одностороннюю альтернативу (increasing / decreasing), но будьте аккуратно, чтобы не ошибиться с направлением альтернативы и не получить противоположный результат

In [ ]:
gq_test = het_goldfeldquandt(lab1.cpi, X, idx = 4, drop = 0.2, alternative = 'two-sided')
stat, p_value, alternative = gq_test[0:3]
print(f'Statistic: {stat}, P-Value: {p_value}, alternative: {alternative}')

Переоценим модель с робастными стандартными ошибками (HC3) - состоятельными в условиях гетероскедастичности - и сравним результаты с предыдущей моделью

Обратите внимание, оценки коэффициентов остаются без изменений, меняются только стандартные ошибки, так как используем специальную матрицу весов HC3 для ошибок

In [ ]:
m1_1 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit(cov_type = "HC3")
print(m1_1.summary())

In [ ]:
comparison = pd.DataFrame({
    'coef_m1': m1.params,
    'se_m1': m1.bse,
    'coef_HC3': m1_1.params,
    'se_HC3': m1_1.bse,
    'se_ratio': m1_1.bse / m1.bse,
    'coef_diff': m1_1.params - m1.params
})

print("Сравнение стандартных ошибок исходных и HC3:")
print(comparison.round(3))

Проверим, есть ли в нашем массиве нетипичные наблюдения. Для начала рассмотрим нетипичные наблюдения по предикторам.

In [ ]:
leverage = m1_1.get_influence().hat_matrix_diag

leverage_data = lab1[leverage > 2*np.mean(leverage)]
leverage_data

Посмотрим, как обстоят дела с outliers (нетипичные наблюдения по y).

In [ ]:
influence = m1_1.get_influence()
student_resid = influence.resid_studentized_external

outlier_data = lab1[np.abs(student_resid) > 3].copy()
outlier_data['student_resid'] = student_resid[np.abs(student_resid) > 3]
outlier_data

Как мы знаем, нетипичные наблюдения только по X или только по Y не меняют значимым образом результаты. Поэтому обратимся к влиятельным наблюдениям - комбинациям нетипичных наблюдений по X и Y. Определим для начала влиятельные наблюдения по мере Кука:

In [ ]:
cooks_dist = m1_1.get_influence().cooks_distance[0]
cooks_dist

lab1['cooks_dist'] = cooks_dist

influential_obs = lab1[cooks_dist > 4 / len(lab1)]

influential_obs_ordered = influential_obs.sort_values('cooks_dist', ascending = False)
influential_obs_ordered

Отдельно выведем значение расстояния Кука и количество наблюдений, превышающих данное значение

In [ ]:
print(f"Порог Cook's D: {4 / len(lab1):.4f}")
print(f"Количество влиятельных наблюдений: {len(influential_obs)}")

Для удобства построим график, отображающий значения потенциала влияния и стьюдентизированных остатков:

In [ ]:
mpl.figure(figsize=(10, 6))

influential = cooks_dist > (4 / len(lab1))

for i in range(len(leverage)):
    if influential[i]:
        mpl.scatter(leverage[i], student_resid[i],
                   s=cooks_dist[i]*1000 + 20,
                   facecolors='none',
                   edgecolors='red',
                   linewidths=2,
                   alpha=0.8)
    else:
        mpl.scatter(leverage[i], student_resid[i],
                   s=cooks_dist[i]*1000 + 20,
                   facecolors='none',
                   edgecolors='gray',
                   linewidths=1,
                   alpha=0.6)

for i in range(len(leverage)):
    if influential[i]:
        country_name = lab1.iloc[i]['country']
        mpl.annotate(country_name,
                    xy=(leverage[i], student_resid[i]),
                    xytext=(8, 8),
                    textcoords='offset points',
                    fontsize=9,
                    fontweight='bold',
                    color='red',
                    alpha=0.8,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                             edgecolor='red', alpha=0.7))

mpl.axhline(y=0, color='black', linestyle='-', alpha=0.3, linewidth=0.5)
mpl.axhline(y=2, color='red', linestyle='--', alpha=0.5, linewidth=1)
mpl.axhline(y=-2, color='red', linestyle='--', alpha=0.5, linewidth=1)

leverage_threshold = 2 * (m1.df_model + 1) / len(lab1)
mpl.axvline(x=leverage_threshold, color='orange', linestyle='--', alpha=0.5, linewidth=1, label='Порог влияния')

x_margin = leverage.max() * 0.15
y_margin = max(abs(student_resid)) * 0.15

mpl.xlim(-x_margin, leverage.max() + x_margin)
mpl.ylim(-max(abs(student_resid)) - y_margin, max(abs(student_resid)) + y_margin)

mpl.xlabel('Потенциал влияния (leverage)', fontsize=12)
mpl.ylabel('Стьюдентизированные остатки', fontsize=12)
mpl.title('Влиятельные наблюдения', fontsize=14)

mpl.grid(True, alpha=0.3)

red_circle = mlines.Line2D([], [], color='red', marker='o', linestyle='None',
                          markersize=8, markeredgewidth=2, markerfacecolor='none',
                          label='Влиятельные наблюдения')
gray_circle = mlines.Line2D([], [], color='gray', marker='o', linestyle='None',
                           markersize=8, markeredgewidth=1, markerfacecolor='none',
                           label='Невлиятельные наблюдения')
mpl.legend(handles=[red_circle, gray_circle], loc='best')

mpl.tight_layout()
mpl.show()

Кроме этого, можно диагностировать наблюдения, влиятельные с точки зрения изменения конкретного параметра. Для иллюстрации рассмотрим коэффициент при ключевом предикторе dem:

In [ ]:
dfbetas = m1_1.get_influence().dfbetas

coef_index = 1
lab1['dfbetas_dem'] = np.abs(dfbetas[:, coef_index])

dfbeta_dem_data = lab1[np.abs(dfbetas[:, coef_index]) > 2 / np.sqrt(len(lab1))]

dfbeta_dem_data_ordered = dfbeta_dem_data.sort_values('dfbetas_dem', ascending = False)
dfbeta_dem_data_ordered